In [29]:
import re
import pandas as pd
import funcs 
import importlib
import unicodecsv as csv
from timeit import default_timer as timer
from datetime import datetime, timedelta

importlib.reload(funcs)

<module 'funcs' from 'C:\\Users\\Mag\\Documents\\_projekty\\praca\\funcs.py'>

## 0. Wczytanie danych zrzuconych z bazy SQL do csv

In [2]:
politicians = pd.read_csv("00_dane/portraits.csv", sep='|')
#speeches = pd.read_csv("00_dane/speech_data.csv", sep='|')

#wczytujemy plik po modyfikacjach w SQL
speeches = pd.read_csv("00_dane/speeches.csv", sep='|')

C:\Users\Mag\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 1. Znalezienie imienia i nazwiska przemawiającego

In [5]:
speeches['author'] = speeches['speech_raw'].apply(funcs.name_finder)

Zapisanie rozszerzonych danych do dalszych analiz.

In [5]:
speeches.to_csv('00_dane/speeches.csv', encoding='UTF=8')

## 2. Czyszczenie tekstu przemówień pt.1


### 2.1. Wczytanie danych po czyszczeniu SQL i dodaniu autora

In [19]:
politicians = pd.read_csv("00_dane/portraits.csv", sep='|')
speeches = pd.read_csv("00_dane/speeches_2.csv", sep='|')

#speeches.to_csv('00_dane/speeches_2.csv', encoding='UTF=8')

C:\Users\Mag\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
len(speeches['speech_raw']), sum(1-pd.isnull(speeches['author_final']))


(272221, 225388)

### 2.2. Czyszczenie znaków specjalnych i wycinanie nagłówka.

In [21]:
start = timer()
speeches['speech_raw'] = speeches['speech_raw'].apply(funcs.rem_head)
speeches['speech_raw'] = speeches['speech_raw'].apply(funcs.clear_text)
speeches['speech_title'] = speeches['speech_title'].apply(funcs.clear_text)
end = timer()
print(end - start)

160.51025696024772


Wycinam fragment, który jest tytułem:
1. Dodaję "\" przed nawiasami w tytule.
2. Wycinam tytuł z treści.

In [22]:
start = timer()
speeches['speech_title'] = speeches['speech_title'].apply(lambda t: re.sub('\(','\(',t))
speeches['speech_title'] = speeches['speech_title'].apply(lambda t: re.sub('\)','\)',t))
print(timedelta(seconds=timer()-start))

0:00:01.926572


Jeden "problematyczny" case.

In [27]:
speeches = speeches[~speeches['speech_raw'].isnull()]
speeches = speeches[~speeches['speech_title'].isnull()]

In [23]:
speeches['speech_title'] = [re.sub('\[COM\\\\\(2005\\\\\)0119 - C6-0099/2005 - 2005/0043\\\\\(COD\\\\\)\]','',t) for t in speeches['speech_title']]
speeches['speech_raw'] = speeches['speech_raw'].apply(funcs.rem_nasty) 

In [30]:
speeches['speech_raw'] = [funcs.rem_title(t, r) for t, r, in zip(speeches['speech_title'], speeches['speech_raw'])]


### 2.3. Zapisanie zmodyfikowanego pliku

In [119]:
start = timer()
speeches.to_csv('00_dane/speeches_2.csv', encoding='UTF-8', index=False, sep="|")
print(timedelta(seconds=timer()-start))

0:00:29.467977


## 3. Łączenie fragmentów przemówień

In [33]:
%%time
sp_joined = speeches.groupby(['date_', 'author_final','speech_title']).agg({'speech_raw': lambda x: ' '.join(x)}).reset_index()

Wall time: 4.46 s


## 4. Czyszczenie tekstu przemówienia pt 3.


### 4.1. Usuwanie bardziej ogólnych elementów
- Wypowiedzi innych posłów i komentarze typu "Oklaski" pojawiające się w nawiasach
- "Poseł Imię Nazwisko:"

In [34]:
start = timer()
sp_joined['speech_raw'] = sp_joined['speech_raw'].apply(lambda t: re.sub('\([\w.,: ]+\)','',t))
sp_joined['speech_raw'] = sp_joined['speech_raw'].apply(lambda t: re.sub('[Pp]oseł ([\w -\.]+):','',t))
sp_joined['speech_raw'] = sp_joined['speech_raw'].apply(lambda t: re.sub('\s+',' ',t))
print(timedelta(seconds=timer()-start))

0:00:51.843886


### 4.2. Usuwanie przeszkadzających elementów -tworze drugą kolumnę
- Panie Marszałku!
- Pani Marszałek!
- Wysoka Izbo!
- Panie Ministrze!
- Dziękuję bardzo.
- Wysoki Sejmie!
- Panie i Panowie Posłowie!
- Panie Posłanki! Panowie Posłowie!
- Wysokie Zgromadzenie Narodowe!

In [37]:
sp_joined['speech_rawer'] = sp_joined['speech_raw'].apply(lambda t: re.sub('Panie Marszałku!','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Szanowny Panie Ministrze!','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Pani Marszałek!','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Wysoka Izbo!','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Panie Ministrze!','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Dziękuję bardzo.','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Wysokie Zgromadzenie Narodowe!','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Wysoki Sejmie!','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Panie Posłanki! Panowie Posłowie!','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Panie i Panowie Posłowie!','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('\s+',' ',t))

sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Dziękuję[.,] panie marszałku[.,]','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Dziękuję.','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Panie!','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('Panowie!','',t))
sp_joined['speech_rawer'] = sp_joined['speech_rawer'].apply(lambda t: re.sub('\s+',' ',t))

Wgranie do pliku

In [38]:
sp_joined.to_csv('00_dane/sp_joined.csv', encoding='UTF-8', index=False, sep="|")